# RFE
RFEは、変数減少法と同じく、最初に全ての特徴量を使ってモデルを構築します。 そのモデルの中で最も重要度の低いを特徴量を削り、性能を再計測するという処理を、指定数(デフォルトでは1)の特徴量になるまで繰り返します。 重要度の指標にはfeature_importancesやcoefが使われます。

【機械学習】RFEを用いた特徴量の選別 https://qiita.com/nanairoGlasses/items/d7d4c190d11ba663635d

- Recursive Feature Elimination　の略。再帰的特徴消去。
- 特徴に重みを割り当てる外部推定機（ランダムフォレストや勾配ブースティング、線形モデルなど）が与えられた時、特徴集合を走査、どの特徴が重要であるかを発見し、指定した特徴数になるまで特徴の消去を行う。
- Wrapper Methodのうちのひとつ

## Wrapper Method
- モデルを実際に訓練することで変数選択をする
- 超遅い。
- 交差検証を使用する。
- 結構良い特徴の部分集合が見つかる。Filter Methodよりも過学習が起きやすい。
- Forward Selection, Backward Elimination, Recursive Feature Eliminationなど


## GradientBoostingRegressor
勾配ブースト回帰


## ブースティング法 
- アンサンブル学習の一つ。
- ベース推定器が順次構築され、結合推定器のバイアスを低減しようとする。 モチベーションはいくつかの弱いモデルを組み合わせて強力なアンサンブルを作り出すことです。例：AdaBoost、勾配ツリーブースト、...

In [1]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, math, warnings
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingRegressor
import pprint
import itertools

In [2]:
every_MCS = 100

In [3]:
#変えた

b = [i / 1000 for i in range(1, 5000, 2)]
print("bのトップバッターは{}である".format(b[0])) #0を目指したかった
print("bのラストは{}である".format(b[-1])) #5を目指したかった
max_step = len(b)
print("max_stepは{}である".format(max_step)) 

bのトップバッターは0.001である
bのラストは4.999である
max_stepは2500である


In [4]:
X = pd.read_table('geneX_df.txt',sep ='\t')# 説明変数（こっちの特徴量上位5つ取り出す）
print("X.shape is {}".format(X.shape))

y = pd.read_table('ERY_df.txt' ,sep ='\t')# 目的変数
#print('type of y is {}'.format(type(y)))

num_sample = X.shape[0]#サンプル数
num_feature = X.shape[1]#特徴量の数

#Kをはじめにlist upするyの数とする

X.shape is (781, 16799)


サンプルが

In [5]:
#falseしかない！！
#X.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int)

In [6]:
X.head()

,ARHGEF10L,HIF3A,RNF17,RNF11,REM1,RTN4RL2,C16orf13,C16orf11,FGFR1OP2,TSKS,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
TCGA-A8-A092-01,-0.92667,-4.836158,-0.545559,-0.425362,-2.511796,1.420118,-0.076931,-0.583953,-0.064954,-1.262546,...,-0.727478,-1.319172,-0.534283,-0.871063,0.137041,-0.494064,-0.396097,-1.178462,-1.748619,-0.659749
TCGA-D8-A1JK-01,-0.83967,-3.260158,0.420041,0.718538,-0.322796,-1.267982,0.252869,-0.583953,-0.031154,-1.634746,...,-0.165278,-1.691372,0.783017,0.037337,-0.051459,0.730136,1.388803,1.339138,-1.924519,0.075451
TCGA-E2-A10C-01,-0.11937,-4.646958,-0.545559,-0.453962,-1.073796,-1.506382,1.084969,1.680847,0.010146,0.890654,...,-0.166078,-1.129972,-0.350883,-0.681864,-0.206159,-0.406364,0.354103,-0.003162,-0.903719,-0.195349
TCGA-BH-A0C7-01,0.28803,-1.599358,-0.545559,-0.211162,-2.041396,-0.156182,-0.861231,-0.583953,-0.249454,-1.029746,...,-0.122478,-0.488172,-0.616583,-1.243263,0.330741,-0.188764,0.420403,-0.456262,0.914881,-0.427849
TCGA-A8-A06N-01,0.61053,-4.268858,-0.545559,0.764638,-2.182996,-1.431682,-0.876131,-0.583953,0.560146,-1.634746,...,-0.429378,2.566228,-1.073483,-1.243263,0.133141,-2.727864,-0.306397,-0.882762,-3.308419,-0.224949


In [7]:
#type(X.iloc[3, 5]) #numpy.float64

In [8]:
y.head()

,ER_Status_nature2012
TCGA-A8-A092-01,Positive
TCGA-D8-A1JK-01,Negative
TCGA-E2-A10C-01,Positive
TCGA-BH-A0C7-01,Positive
TCGA-A8-A06N-01,Positive


In [9]:
len(y)

781

In [10]:
# 1 行目の 1 列目の値を取得
#df.iloc[1,1]

for i in range(len(y)):
    if y.iloc[i, 0]=="Positive":
        y.iloc[i, 0]=np.float(1)
    else:
        y.iloc[i, 0]=np.float(0)

In [11]:
y.head()

,ER_Status_nature2012
TCGA-A8-A092-01,1
TCGA-D8-A1JK-01,0
TCGA-E2-A10C-01,1
TCGA-BH-A0C7-01,1
TCGA-A8-A06N-01,1


In [12]:
type(y)

pandas.core.frame.DataFrame

In [13]:
series_y = y.loc[:, 'ER_Status_nature2012']
series_y

TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
                  ..
TCGA-EW-A1OX-01    1
TCGA-BH-A0BJ-01    1
TCGA-EW-A1IY-01    1
TCGA-D8-A142-01    0
TCGA-BH-A1FC-01    0
Name: ER_Status_nature2012, Length: 781, dtype: int64

In [14]:
type(series_y)

pandas.core.series.Series

In [15]:
num_sample = X.shape[0]#サンプル数
num_feature = X.shape[1]#特徴量の数

In [16]:
K = 2*210

### 特徴量選択

```
'''その一　Filter method'''
#https://aotamasaki.hatenablog.com/entry/2018/04/27/222536

#mask_oriの計算（以降のエネルギー計算の基準となる）
#ori_y == yとなる場合だけ、E(y)==0となる仕様

selector = SelectKBest(k=num_select) #Filter method #単変量選択（分類：chi2、f_classif、mutual_info_classif）
selector.fit(X, series_y)
mask_ori = selector.get_support() #各特徴量を選択したか否かのmaskを取得


'''その二　Wrapper method'''
#recursive feature eliminationを用いた選択
#スコア算出ができるので特徴量数を指定できるかもしれない
select = RFECV(LogisticRegression(), cv=10, scoring='average_precision')
select.fit(X, y)
mask = select.support_


'''その三　Boruta：randomForestを用いた変数選択法'''
#https://www.google.co.jp/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&ved=0ahUKEwiEzMP4p9naAhWBk5QKHbRjC9oQFggoMAA&url=https%3A%2F%2Fwww.jstatsoft.org%2Farticle%2Fview%2Fv036i11%2Fv36i11.pdf&usg=AOvVaw3tyiHN0BCe2fkkAA6xEVDE

forest = RandomForestClassifier()
# define Boruta feature selection method
select = boruta_py.BorutaPy(forest, n_estimators=10)
select.fit(X, y)
mask = select.support_
```

In [17]:
# RFEモデルの生成
# 外部推定機には勾配ブースティングを指定。目標特徴量は２０に設定
rfe = RFE(estimator=GradientBoostingRegressor(random_state=0), n_features_to_select=20, step=0.5)

# 特徴量削減の実行
rfe.fit(X, series_y.values.ravel())

mask_ori = rfe.support_

In [18]:
series_y

TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
                  ..
TCGA-EW-A1OX-01    1
TCGA-BH-A0BJ-01    1
TCGA-EW-A1IY-01    1
TCGA-D8-A142-01    0
TCGA-BH-A1FC-01    0
Name: ER_Status_nature2012, Length: 781, dtype: int64

In [19]:
mask_ori

array([False, False, False, ..., False, False, False])

In [20]:
#エネルギーが0(最低)となるのがいつか基準を先に計算しておきましょう
def E(y): #毎回のmaskとyが引数、合致度合いの計算を行う
    #selector = SelectKBest(score_func=chi2, k=num_select) #SelectKBest #chi2...分類
    rfe = RFE(estimator=GradientBoostingRegressor(random_state=0), n_features_to_select=20, step=0.5)
    rfe.fit(X, y.values.ravel())
    mask_each = rfe.support_
    
    energy_trait_sum = 1 #0を回避するため
    for i in range(len(mask_each)):
        if mask_each[i]!=mask_ori[i]:
            energy_trait_sum += 1
    return energy_trait_sum #もとのyのときのmask(これをmaskとしている)との合致度合い（大きい⇒よく合致している）

In [21]:
E(series_y)

1

In [22]:
series_y

TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
                  ..
TCGA-EW-A1OX-01    1
TCGA-BH-A0BJ-01    1
TCGA-EW-A1IY-01    1
TCGA-D8-A142-01    0
TCGA-BH-A1FC-01    0
Name: ER_Status_nature2012, Length: 781, dtype: int64

👇ここで毎回```The kernel appears to have died. It will restart automatically.```
- num_sample = 10 できた
- num_sample = 100　もうだめ
- num_sample = 50　これでもだめ

### 今日教わったこと、わかったこと
```itertools.product```禁止

- そもそもモンテカルロはエネルギー差を見るもの。１つだけ変えるからエネルギー差がわかる

tried👇but didn't succeed

頻繁に起こるエラー
```'int' object is not subscriptable```。箇所は```ori_yj = y[j]```。```y```は```int```ではないのになぜ。

In [23]:
series_y #一行前までseriesなのになぜか数字になる時がある。

TCGA-A8-A092-01    1
TCGA-D8-A1JK-01    0
TCGA-E2-A10C-01    1
TCGA-BH-A0C7-01    1
TCGA-A8-A06N-01    1
                  ..
TCGA-EW-A1OX-01    1
TCGA-BH-A0BJ-01    1
TCGA-EW-A1IY-01    1
TCGA-D8-A142-01    0
TCGA-BH-A1FC-01    0
Name: ER_Status_nature2012, Length: 781, dtype: int64

In [ ]:
Ene = [] #これからi(温度)ごとに入れていく。一行に入っているのもlistにしたい(行i, 列k, 要素Ene)
y_list = [] #これからi(温度)ごとに、選択したyを記録していく。(行i, 要素num_tl)

Ene.append({})
y_list.append([])
for k in range(K):
    Ene[-1][k]= 1
    y_list[-1].append(series_y)
    
for i in range(1, max_step):   
    Ene.append({})
    y_list.append([])
    for k in range(K): #一度に熟すサンプル数
        #MCS
        #𝑃(B)/𝑃(A)>random ならば状態Bを採用し，そうでなければ状態Aを採用(Bを先、Aを過去とする)
        j = random.randrange(num_sample) #何番目の0 or 1をいじるか決める
        ori_yj = series_y[j]
        series_y[j] = 1-series_y[j]
        Ene[-1][k]= E(pd.Series(series_y))
        y_list[-1].append(series_y)
        
        #今回のターンはiね。
        r = math.exp(-Ene[i][k]*b[i]-(-Ene[i-1][k]*b[i-1])) #Ene[i-1][k]*b[i-1] - Ene[i][k]*b[i] #大きいほうが良い
        #rr = math.exp(-eene[0][0]*b[3]-(-eene[1][0]*b[2])) 
        R = np.random.uniform(0.0, 1.0)
        if R > r : #乱数R > エネルギー差の大きさを用いた計算
            Ene[-1][k] = Ene[i-1][k]
            y_list[-1][k] = y_list[i-1][k]
            series_y = y_list[i-1][k]
    
    
    #Eneのi行目に数値が全部入って、かつ👇を満たすようだったら
    if i%every_MCS == 0:
        Ei_sorted = sorted(Ene[-1].items(), key=lambda x:x[1]) #辞書をvalueを元にsort
        tuple_kE_list = Ei_sorted[0:int(len(Ei_sorted))] #tuple(k, Ene)
        
        #y_list[-1](y_listの最後尾)を並べ替え
        y_list_sort = [y_list[-1][tuple_kE_list[k][0]] for k in range(K)]
        
        #ここまでで、Ei_sorted, tuple_kE_list, y_list_sortによってsortedされたEneとy_listが得られた
        
        #Ene[i][k]はいつでも0以外にしないとエラーが起こる
        sumQ = sum([math.exp(-(b[i]-b[i-1])*Ene[i][k]) for k in range(K)])
        
        #すべてのアイテムについてコピー可能数を計算する
        tbi = [K*math.exp(-(b[i]-b[i-1])*tuple_kE_list[k][1])/sumQ for k in range(K)]
        
        next_K = 0 #次のKの値を決めよう
        reserve_min_num = 0
        for k in range(K):
            if next_K <= K:
                if tbi[k] % 1 >= 0.5:
                    tbi[k] = tbi[k] // 1 +1
                else:
                    tbi[k] = tbi[k] // 1
                    
                next_K += tbi[k]
            else:
                reserve_min_num = k-1
                break #forループを抜ける
        
        
        #k, k+1, ..., Kにおいてtuple_kE_list[k][1]を0, 1, ...k-1のととって変える
        for k in range(reserve_min_num): #次にも引き継がれるk
            for kk in range(tbi[k]): #kによって含まれる要素数が異なる
                ll = tbi[k]*k + kk + 1 #エネルギー大きいほうからll番目
                ori_k = tuple_kE_list[-ll][0]#いわゆるkを知る
                y_list[-1][ori_k] = y_list_sort[-ll]
                Ene[-1][ori_k] = tuple_kE_list[-ll][1]
                
        K = int(next_K)
                

if len(Ene) == max_step:               
    #初期値を抜く
    del Ene[0]
    del y_list[0]

In [ ]:
pd.DataFrame(Ene)

In [ ]:
Ene_v_sum_list = []
Ene_v_sum_list_append = Ene_v_sum_list.append
for each_Ene in Ene:
    Ene_v_sum = sum([Ene_v for Ene_v in each_Ene.values()])
    Ene_v_sum_list_append(Ene_v_sum/K)
#Ene_v_sum_list
plt.plot(Ene_v_sum_list)
plt.savefig('meeting_0428_0_figure_0.png')
print(Ene_v_sum_list[-1])

In [ ]:
#毎度(約)K個あるアイテムの内、0, 100, 200, 400番目のアイテムのエネルギーの移り変わりをplotする
ene0 = []
ene100 = []
ene200 = []
ene400 = []
for i in range(len(Ene)):
    ene0.append(Ene[i][0])
    ene100.append(Ene[i][100])
    ene200.append(Ene[i][200])
    ene400.append(Ene[i][400])

plt.plot(ene0)
plt.plot(ene100)
plt.plot(ene200)
plt.plot(ene400)
plt.savefig('meeting_0428_0_figure_1.png')

In [ ]:
#エネルギーが最小(1)のアイテム数の移り変わりをplotする

ene0_num = []
for i in range(len(Ene)):
    ene0_num.append(len([k for k, v in Ene[i].items() if v == 1]))
    
plt.plot(ene0_num)
plt.savefig('meeting_0428_0_figure_2.png')